In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import os
from PIL import Image
import cv2
import re    

In [2]:
# parametros notebook
DIM = 30
seed = 42
np.random.seed(42)
tf.random.set_seed(42)

In [39]:
# planteamos distintas pruebas

#n_pca: cantidad de componentes principales a considerar
#excl_n_prim_comp: excluir las primeras excl_n_prim_comp componenetes principales (0 no se excluye ninguna)
#nueronas_layer_1: neuronas capa oculta 1
#nueronas_layer_2: neuronas capa oculta 2
#n_epochs: cantidad de epochs... NO SE TOMA EN CUENTA... SE HARDCODEA MÁS ABAJO!!!

# PARA LA ULTIMA PARTE DE LA NOTEBOOK ES IMPORTANTE TENER A MANO LOS PARAMETROS
# "n_pca" y "excl_n_prim_comp" del modelo seleccionado!!!

pruebas = {
    #"prueba_1": {"n_pca":100, "excl_n_prim_comp":5, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    #"prueba_2": {"n_pca":150, "excl_n_prim_comp":5, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    #"prueba_3": {"n_pca":170, "excl_n_prim_comp":5, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    #"prueba_4": {"n_pca":50, "excl_n_prim_comp":5, "nueronas_layer_1":36, "nueronas_layer_2":22, "n_epochs":100},
    "prueba_4_1": {"n_pca":60, "excl_n_prim_comp":5, "nueronas_layer_1":40, "nueronas_layer_2":25, "n_epochs":100},
    #"prueba_4_2": {"n_pca":40, "excl_n_prim_comp":3, "nueronas_layer_1":32, "nueronas_layer_2":22, "n_epochs":100},
    #"prueba_5": {"n_pca":100, "excl_n_prim_comp":4, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    #"prueba_6": {"n_pca":100, "excl_n_prim_comp":3, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    #"prueba_7": {"n_pca":100, "excl_n_prim_comp":2, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    #"prueba_8": {"n_pca":100, "excl_n_prim_comp":0, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    #"prueba_9": {"n_pca":100, "excl_n_prim_comp":5, "nueronas_layer_1":60, "nueronas_layer_2":40, "n_epochs":100},
    #"prueba_10": {"n_pca":100, "excl_n_prim_comp":5, "nueronas_layer_1":60, "nueronas_layer_2":20, "n_epochs":100},
    #"prueba_11": {"n_pca":100, "excl_n_prim_comp":5, "nueronas_layer_1":50, "nueronas_layer_2":30, "n_epochs":100},
    "prueba_12": {"n_pca":100, "excl_n_prim_comp":0, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    "prueba_13": {"n_pca":100, "excl_n_prim_comp":0, "nueronas_layer_1":45, "nueronas_layer_2":26, "n_epochs":100},
    "prueba_14": {"n_pca":100, "excl_n_prim_comp":3, "nueronas_layer_1":45, "nueronas_layer_2":26, "n_epochs":100},
    #"prueba_14": {"n_pca":60, "excl_n_prim_comp":0, "nueronas_layer_1":40, "nueronas_layer_2":25, "n_epochs":100},
    #"prueba_15": {"n_pca":60, "excl_n_prim_comp":0, "nueronas_layer_1":36, "nueronas_layer_2":22, "n_epochs":100},
    #"prueba_16": {"n_pca":40, "excl_n_prim_comp":0, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    #"prueba_17": {"n_pca":100, "excl_n_prim_comp":0, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    #"prueba_18": {"n_pca":100, "excl_n_prim_comp":0, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    "prueba_19": {"n_pca":100, "excl_n_prim_comp":0, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
    "prueba_20": {"n_pca":100, "excl_n_prim_comp":3, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":100},
}

## FUNCIONES DE LA NOTEBOOK (podrian ir en un script)

In [4]:
# defino funciones utiles
def eliminar_numeros(texto):
    return re.sub(r'\d+', '', texto)

In [5]:
## defino funcion para cortar caras de una imagen

import os
import cv2
import face_recognition
from PIL import Image

def cortar_imagenes(input_dir, output_dir, dim=DIM):
    # Cargamos el detector de rostros de la libreria face_recognition
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    i = 0
    # Para cada archivo del directorio de imput comenzamos el loop
    for filename in os.listdir(input_dir):
        i += 1
        # Cargamos la imagen
        input_path = os.path.join(input_dir, filename)
        img = cv2.imread(input_path)

        # Detectamos el rostro en la imagen
        face_locations = face_recognition.face_locations(img)
        
        # Cortamos y cambiamos a escala de grises
        for (top, right, bottom, left) in face_locations:
            face = img[top:bottom, left:right]
            face = cv2.resize(face, (dim, dim))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Guardamos el proceso en la carpeta de salida
            output_path = os.path.join(output_dir, f"{filename}")
            cv2.imwrite(output_path, face)
                

ModuleNotFoundError: No module named 'face_recognition'

In [6]:
# defino funcion para generar nuevas caras
# Generar imagenes aleatorias a partir de imagenes existentes
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def gen_new_image(folder_path, prefix, cantidad_imagenes):

    # obtiene nombre de los archivos con las imagenes
    file_names = os.listdir(folder_path)

    # itera sobre los archivos
    for file_name in file_names:
        # arma ruta a la imagen
        file_path = os.path.join(folder_path, file_name)

        # carga la imagen
        img = image.load_img(file_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # crea un generador de datos con aumentos
        datagen = ImageDataGenerator(
            rotation_range=3,
            width_shift_range=0.025,
            height_shift_range=0.025,
            shear_range=0.025,
            zoom_range=0.025,
            horizontal_flip=True,
            fill_mode="nearest",
        )

        # separa nombre archivo y extension (a usar en el nombre de la nueva imagen generada)
        name, ext = os.path.splitext(file_name)

        # inicializa el bucle para las 'cantidad_imagenes' a generar
        i = 0
        for batch in datagen.flow(x, batch_size=1):
            # define nombre de la nueva imagen generada segun prefijo
            new_filename = f"{name}_{prefix}_{i}{ext}"

            # guarda la imagen aumentada
            new_file_path = os.path.join(folder_path, new_filename)
            img_augmented = image.array_to_img(batch[0])
            img_augmented.save(new_file_path)
            i += 1
            if i >= cantidad_imagenes:
                break


In [7]:
# defino funcion para entrenar 
def run_keras_model(n_pca, excl_n_prim_comp, nueronas_layer_1, nueronas_layer_2, n_epochs, imagenes, nombres, nuevas_imagenes, nuevos_nombres):
    
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(imagenes, nombres, test_size=0.05, random_state=42, stratify=nombres)

    # Aplicar PCA
    pca = PCA(n_components=n_pca, random_state=seed)
    X_train_pca = pca.fit_transform(X_train)
    #X_test_pca = pca.transform(X_test)
    
    # Escalar los datos 
    scaler = StandardScaler()
    #scaler = MinMaxScaler()
    X_train_pca_scaled = scaler.fit_transform(X_train_pca)
    #X_test_pca_scaled = scaler.transform(X_test_pca)
    
    
    # Codificar las etiquetas
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    #y_test_encoded = encoder.transform(y_test)
    y_train_categorical = to_categorical(y_train_encoded, num_classes=18)
    #y_test_categorical = to_categorical(y_test_encoded, num_classes=18)
    
    
    # Definir la red neuronal
    entreno_con = X_train_pca_scaled[:,excl_n_prim_comp:]
    #testeo_con = X_test_pca_scaled[:,excl_n_prim_comp:]
    model = Sequential()
    model.add(Input(shape=entreno_con.shape[1:]))  # Definir la entrada del modelo
    model.add(Dense(nueronas_layer_1, activation='sigmoid'))
    model.add(Dense(nueronas_layer_2, activation='sigmoid'))
    model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

    # Compilar el modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo
    model.fit(entreno_con, y_train_categorical, epochs=n_epochs, batch_size=10, validation_split=0.3)

    # Evaluar el modelo
    #loss, accuracy = model.evaluate(testeo_con, y_test_categorical)
    #print(f'Precisión en el conjunto de prueba: {accuracy * 100:.2f}%')
    
    
    # PREDICCIONES
    # 2. Aplicar PCA
    new_images_pca = pca.transform(nuevas_imagenes)
    new_images_pca_scaled = scaler.transform(new_images_pca)
    evaluo_con = new_images_pca_scaled[:,excl_n_prim_comp:]

    # 3. Hacer predicciones
    predictions = model.predict(evaluo_con)

    # Obtener los nombres correspondientes a las clases predichas
    predicted_names = encoder.inverse_transform(np.argmax(predictions, axis=1))

    # Imprimir las predicciones
    total_predict = len(nuevos_nombres)
    total_correcto = 0
    for real, pred in zip(nuevos_nombres, predicted_names):
        if real == pred:
            total_correcto += 1
    
    # devuelve resultado corrida
    resultado = total_correcto/total_predict
    return resultado, model

In [8]:
# ejemplo de uso
#cortar_imagenes(input_dir="fotos/Recorte manual", output_dir="fotos/Recorte manual otutput 35")
# esta gen_new_imagen usarla sobre la carpeta de salida de cortar_imagenes
#gen_new_image(folder_path="fotos/Recorte manual otutput 35", prefix="gen_aut", cantidad_imagenes=10)

## LEER IMAGENES PARA ENTRENAMIENTO 

In [9]:
# Configuración inicial
dim_imagenes = DIM
data_dir = "fotos_probamos_distintas_opciones/entrenamiento"
nombres = []
imagenes = []

# Cargar imágenes y etiquetas
for archivo in os.listdir(data_dir):
    if archivo.endswith('.jpeg') or archivo.endswith('.jpg'):
        nombre = archivo.split('_')[0].replace(".jpg","").replace(".jpeg","")
        nombre = eliminar_numeros(nombre)
        ruta_imagen = os.path.join(data_dir, archivo)
        imagen = Image.open(ruta_imagen)
        imagen = np.array(imagen.resize((dim_imagenes, dim_imagenes)))  # Redimensionar imágenes para un tamaño uniforme
        if len(imagen.shape) == 3 and imagen.shape[2] == 3:
            imagen = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)  # Convertir a escala de grises si es necesario
        imagenes.append(imagen.flatten())
        nombres.append(nombre)

imagenes_all = np.array(imagenes)
nombres_all = np.array(nombres)

# estand
imagenes_all = imagenes_all/255.0

In [13]:
print(imagenes_all.shape)
print(nombres_all.shape)


(10261, 900)
(10261,)


## LEER IMAGENES PARA PREDICT

In [14]:
# Configuración inicial
data_dir = ("fotos_probamos_distintas_opciones/predict")  # Cambia esto a la ruta de tu directorio de imágenes
nombres = []
imagenes = []

# Cargar imágenes y etiquetas
for archivo in os.listdir(data_dir):
    if archivo.endswith('.jpeg') or archivo.endswith('.jpg'):
        nombre = archivo.split('_')[0].replace(".jpg","").replace(".jpeg","")
        nombre = eliminar_numeros(nombre)
        ruta_imagen = os.path.join(data_dir, archivo)
        imagen = Image.open(ruta_imagen)
        imagen = np.array(imagen.resize((dim_imagenes, dim_imagenes)))  # Redimensionar imágenes para un tamaño uniforme
        if len(imagen.shape) == 3 and imagen.shape[2] == 3:
            imagen = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)  # Convertir a escala de grises si es necesario
        imagenes.append(imagen.flatten())
        nombres.append(nombre)

new_images = np.array(imagenes)
nombres_new = np.array(nombres)

# estand
new_images = new_images/255.0

In [15]:
print(new_images.shape)
print(nombres_new.shape)


(30, 900)
(30,)


## ENTRENAMIENTO

In [16]:
# PRUEBAS CON epochs 500 # SI TARDA MUCHO PROBAR CON 250...
resutados = {}
modelos_500 = {}
for prueba, params in pruebas.items():
    print(f"{prueba}: ")
    resultado, modelo = run_keras_model(n_pca=params.get('n_pca'), 
                    excl_n_prim_comp=params.get('excl_n_prim_comp'), 
                    nueronas_layer_1=params.get('nueronas_layer_1'),
                    nueronas_layer_2=params.get('nueronas_layer_2'),
                    n_epochs=50, # NO TOMA EL QUE DEFINIMOS EN PRUEBAS<<<<<<<<<<<<--------- epochs cambiar aca
                    imagenes=imagenes_all,
                    nombres=nombres_all,
                    nuevas_imagenes=new_images,
                    nuevos_nombres=nombres_new
                    )
    resutados[prueba]=resultado
    modelos_500[prueba]=modelo
resutados

prueba_4_1: 
Epoch 1/50
683/683 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1246 - loss: 2.8398 - val_accuracy: 0.2885 - val_loss: 2.5135
Epoch 2/50
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3262 - loss: 2.3686 - val_accuracy: 0.4291 - val_loss: 1.9988
Epoch 3/50
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4859 - loss: 1.8752 - val_accuracy: 0.5768 - val_loss: 1.6011
Epoch 4/50
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6262 - loss: 1.4968 - val_accuracy: 0.6492 - val_loss: 1.3217
Epoch 5/50
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6913 - loss: 1.2328 - val_accuracy: 0.6882 - val_loss: 1.1414
Epoch 6/50
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7263 - loss: 1.0563 - val_accuracy: 0.7200 - val_loss: 1.0178
Epoch 7/50
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7591 - loss: 0.9302 - val_accuracy: 0.7398 - val_loss: 0.9253
Epoch 8/50
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7773 - loss: 0.8329 - val

{'prueba_4_1': 0.5,
 'prueba_12': 0.4666666666666667,
 'prueba_13': 0.7333333333333333,
 'prueba_14': 0.7333333333333333}

In [40]:
# PRUEBAS CON epochs 200
resutados = {}
modelos_200 = {}
for prueba, params in pruebas.items():
    print(f"{prueba}: ")
    resultado, modelo = run_keras_model(n_pca=params.get('n_pca'), 
                    excl_n_prim_comp=params.get('excl_n_prim_comp'), 
                    nueronas_layer_1=params.get('nueronas_layer_1'),
                    nueronas_layer_2=params.get('nueronas_layer_2'),
                    n_epochs=200, # NO TOMA EL QUE DEFINIMOS EN PRUEBAS <<<<<--------- epochs cambiar aca
                    imagenes=imagenes_all,
                    nombres=nombres_all,
                    nuevas_imagenes=new_images,
                    nuevos_nombres=nombres_new
                    )
    resutados[prueba]=resultado
    modelos_200[prueba]=modelo
resutados

prueba_4_1: 
Epoch 1/200
683/683 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1052 - loss: 2.8615 - val_accuracy: 0.2708 - val_loss: 2.5692
Epoch 2/200
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3585 - loss: 2.4368 - val_accuracy: 0.5060 - val_loss: 2.0292
Epoch 3/200
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5321 - loss: 1.9004 - val_accuracy: 0.6065 - val_loss: 1.6085
Epoch 4/200
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6248 - loss: 1.4971 - val_accuracy: 0.6674 - val_loss: 1.3241
Epoch 5/200
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6896 - loss: 1.2223 - val_accuracy: 0.7026 - val_loss: 1.1359
Epoch 6/200
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7368 - loss: 1.0372 - val_accuracy: 0.7296 - val_loss: 1.0053
Epoch 7/200
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7608 - loss: 0.9058 - val_accuracy: 0.7491 - val_loss: 0.9063
Epoch 8/200
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7829 - loss: 0.80

{'prueba_4_1': 0.75,
 'prueba_12': 0.75,
 'prueba_13': 0.25,
 'prueba_14': 0.75,
 'prueba_19': 0.75,
 'prueba_20': 0.75}

## Pruebo predict

In [19]:
# si cargo nuevas fotos es importante correr esta parte (con True en lugar de False)
# hacer check manual que TOME BIEN LAS CARAS (puede pasar que recorte algo que NO es una cara)
if False:
    cortar_imagenes(input_dir="fotos_probamos_distintas_opciones/probar_nuevas_fotos_input", output_dir="fotos_probamos_distintas_opciones/probar_nuevas_fotos_output")

In [20]:
# importo imagenes predict 2
data_dir = ("fotos_probamos_distintas_opciones/probar_nuevas_fotos_output")  # Cambia esto a la ruta de tu directorio de imágenes
nombres = []
imagenes = []

# Cargar imágenes y etiquetas
for archivo in os.listdir(data_dir):
    if archivo.endswith('.jpeg') or archivo.endswith('.jpg'):
        nombre = archivo.split('_')[0].replace(".jpg","").replace(".jpeg","")
        nombre = eliminar_numeros(nombre)
        ruta_imagen = os.path.join(data_dir, archivo)
        imagen = Image.open(ruta_imagen)
        imagen = np.array(imagen.resize((dim_imagenes, dim_imagenes)))  # Redimensionar imágenes para un tamaño uniforme
        if len(imagen.shape) == 3 and imagen.shape[2] == 3:
            imagen = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)  # Convertir a escala de grises si es necesario
        imagenes.append(imagen.flatten())
        nombres.append(nombre)

new_images = np.array(imagenes)
nombres_new = np.array(nombres)

# estand
new_images = new_images/255.0
print(new_images.shape)
print(nombres_new.shape)


(4, 900)
(4,)


In [21]:
len(nombres_new)

4

In [22]:
def get_preds_predict2(model, n_pca, excl_n_prim_comp, imagenes, nombres, nuevas_imagenes, nuevos_nombres):
    
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(imagenes, nombres, test_size=0.05, random_state=42, stratify=nombres)

    # Aplicar PCA
    pca = PCA(n_components=n_pca, random_state=12)
    X_train_pca = pca.fit_transform(X_train)
    #X_test_pca = pca.transform(X_test)
    
    # Escalar los datos 
    scaler = StandardScaler()
    #scaler = MinMaxScaler()
    X_train_pca_scaled = scaler.fit_transform(X_train_pca)
    #X_test_pca_scaled = scaler.transform(X_test_pca)
    
    
    # Codificar las etiquetas
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    #y_test_encoded = encoder.transform(y_test)
    y_train_categorical = to_categorical(y_train_encoded, num_classes=18)
    #y_test_categorical = to_categorical(y_test_encoded, num_classes=18)
    
    
    # Definir la red neuronal
    entreno_con = X_train_pca_scaled[:,excl_n_prim_comp:]
    #testeo_con = X_test_pca_scaled[:,excl_n_prim_comp:]    
    
    # PREDICCIONES
    # 2. Aplicar PCA
    new_images_pca = pca.transform(nuevas_imagenes)
    new_images_pca_scaled = scaler.transform(new_images_pca)
    evaluo_con = new_images_pca_scaled[:,excl_n_prim_comp:]

    # 3. Hacer predicciones
    predictions = model.predict(evaluo_con)

    # Obtener los nombres correspondientes a las clases predichas
    predicted_names = encoder.inverse_transform(np.argmax(predictions, axis=1))

    # Imprimir las predicciones
    total_predict = len(nuevos_nombres)
    total_correcto = 0
    for real, pred in zip(nuevos_nombres, predicted_names):
        if real == pred:
            total_correcto += 1
         
    # df predicciones        
    df_preds = pd.DataFrame(predictions.round(2))
    class_names = encoder.classes_
    df_preds.columns = class_names
    
    # devuelve resultado corrida
    resultado = total_correcto/total_predict
    return resultado, df_preds

In [23]:
resultado13, df_preds13 = get_preds_predict2(
    model=modelos_500["prueba_13"], 
    n_pca=100, 
    excl_n_prim_comp=0, 
    imagenes=imagenes_all, 
    nombres=nombres_all, 
    nuevas_imagenes=new_images, 
    nuevos_nombres=nombres_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


In [24]:
resultado13

0.5

In [25]:
df_preds13

,carlos,federicoG,federicoR,florencia,francoA,francoS,gerard,gustavo,joaquin,juan,lautaro,lisandro,marco,matias,natalia,noelia,paola,victorio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.00,0.0,0.96,0.0,0.0,0.0,0.0,0.00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.86,0.0,0.10,0.0,0.0,0.0,0.0,0.04
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,1.0,0.00,0.0,0.0,0.0,0.0,0.00


In [26]:
resultado14, df_preds14 = get_preds_predict2(
    model=modelos_500["prueba_14"], 
    n_pca=100, 
    excl_n_prim_comp=3, 
    imagenes=imagenes_all, 
    nombres=nombres_all, 
    nuevas_imagenes=new_images, 
    nuevos_nombres=nombres_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [28]:
resultado14

0.75

In [29]:
df_preds14

,carlos,federicoG,federicoR,florencia,francoA,francoS,gerard,gustavo,joaquin,juan,lautaro,lisandro,marco,matias,natalia,noelia,paola,victorio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.34,0.27,0.0,0.0,0.0,0.38,0.0


In [31]:
resultado14, df_preds14 = get_preds_predict2(
    model=modelos_200["prueba_14"], 
    n_pca=100,              #<<<---- tiene que tener mismos valores que el experimento
    excl_n_prim_comp=3,     #<<<---- tiene que tener mismos valores que el experimento
    imagenes=imagenes_all, 
    nombres=nombres_all, 
    nuevas_imagenes=new_images, 
    nuevos_nombres=nombres_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [32]:
resultado14

0.75

In [35]:
resultado13, df_preds13 = get_preds_predict2(
    model=modelos_200["prueba_13"], 
    n_pca=100,          #<<<---- tiene que tener mismos valores que el experimento
    excl_n_prim_comp=0, #<<<---- tiene que tener mismos valores que el experimento
    imagenes=imagenes_all, 
    nombres=nombres_all, 
    nuevas_imagenes=new_images, 
    nuevos_nombres=nombres_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [36]:
resultado13

0.75

In [37]:
df_preds13

,carlos,federicoG,federicoR,florencia,francoA,francoS,gerard,gustavo,joaquin,juan,lautaro,lisandro,marco,matias,natalia,noelia,paola,victorio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.00,0.0,0.0,0.95,0.00,0.00,0.00,0.0,0.00,0.0,0.00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.83,0.12,0.00,0.01,0.0,0.02,0.0,0.02
2,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.99,0.00,0.00,0.00,0.0,0.00,0.0,0.00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.0,0.00,0.56,0.43,0.00,0.0,0.00,0.0,0.00


In [41]:
resultado19, df_preds19 = get_preds_predict2(
    model=modelos_200["prueba_19"], 
    n_pca=100,          #<<<---- tiene que tener mismos valores que el experimento
    excl_n_prim_comp=0, #<<<---- tiene que tener mismos valores que el experimento
    imagenes=imagenes_all, 
    nombres=nombres_all, 
    nuevas_imagenes=new_images, 
    nuevos_nombres=nombres_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [42]:
resultado19

0.75

In [43]:
df_preds19

,carlos,federicoG,federicoR,florencia,francoA,francoS,gerard,gustavo,joaquin,juan,lautaro,lisandro,marco,matias,natalia,noelia,paola,victorio
0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.79,0.01,0.19,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.04,0.95,0.0,0.0,0.0,0.0,0.0


In [44]:
resultado20, df_preds20 = get_preds_predict2(
    model=modelos_200["prueba_20"], 
    n_pca=100,          #<<<---- tiene que tener mismos valores que el experimento
    excl_n_prim_comp=3, #<<<---- tiene que tener mismos valores que el experimento
    imagenes=imagenes_all, 
    nombres=nombres_all, 
    nuevas_imagenes=new_images, 
    nuevos_nombres=nombres_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [45]:
resultado20

0.75

In [46]:
df_preds20

,carlos,federicoG,federicoR,florencia,francoA,francoS,gerard,gustavo,joaquin,juan,lautaro,lisandro,marco,matias,natalia,noelia,paola,victorio
0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.97,0.0,0.01,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.49,0.0,0.0,0.0,0.5,0.0
